In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
%load_ext memory_profiler

In [3]:
% memit

peak memory: 264.23 MiB, increment: 0.08 MiB


<font color=blue size=4><b> 보이스데이터->분석용데이터 feature 추출</b></font>
- logspec = librosa.amplitude_to_db(melspec) - librosa 함수 수정
- fn 파일 경로 구분자 수정
- 각 폴더 내 각 레이블 데이터 모두 포함 ( 0 ~ N ) - N = 레이블 수
- 현재 N => 여, 남, 소, 남노, 여노 = 5

http://egloos.zum.com/nemonein/v/5326841
rename code


rename 's/name1/name2/' targetfile

In [4]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*",bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            fn = fn.replace('\\','/')
            sound_clip,s = librosa.load(fn)
#            print(fn)
            label = int(fn.split('/')[3].split('_')[0])
        
#            print(label)
            for (start,end) in windows(sound_clip,window_size):
                #print("-w", window_size)
                start = int(start)
                end = int(end)
                #print("-s",int(len(sound_clip[start:end])))
                if(len(sound_clip[start:end]) == int(window_size)):
                    signal = sound_clip[start:end]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.amplitude_to_db(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features), np.array(labels,dtype = np.int)

def extract_feature_array(filename, bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    sound_clip,s = librosa.load(filename)        
    for (start,end) in windows(sound_clip,window_size):
        start = int(start)
        end = int(end)
        if(len(sound_clip[start:end]) == window_size):
            signal = sound_clip[start:end]
            melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
            logspec = librosa.amplitude_to_db(melspec)
            logspec = logspec.T.flatten()[:, np.newaxis].T
            log_specgrams.append(logspec)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

### 음성 -> npy 파일 생성
- train, valid, test용 audio npy 파일 생성
- 훈련용 데이터 셋을 폴더 단위로 저장
- 현재 트레이닝 데이터셋은 1개 폴더로 구성
- train, valid, test_fold_name 은 사용자 편의에 맞게 명명

In [5]:
def save_folds(data_dir):
    for k in range(0,3):
        fold_name = fold_n[k]
        print ("\nSaving " + fold_name)
        features, labels = extract_features(parent_dir, [fold_name])
        labels = one_hot_encode(labels)
        
        print ("Features of", fold_name , " = ", features.shape)
        print ("Labels of", fold_name , " = ", labels.shape)
        
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        np.save(feature_file, features)
        print ("Saved " + feature_file)
        np.save(labels_file, labels)
        print ("Saved " + labels_file)

def assure_path_exists(path):
    mydir = os.path.join(os.getcwd(), path)
    if not os.path.exists(mydir):
        os.makedirs(mydir)

### npz파일로부터 feature, label 정보 로드

In [6]:
def load_all_folds(): 
    subsequent_fold = False
    for k in range(0,1):
        fold_name = 'fold' + str(k)
        #fold_name = fold_n[k]
        print ("\nAdding " + fold_name)
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        loaded_features = np.load(feature_file)
        loaded_labels = np.load(labels_file)
        print ("New Features: ", loaded_features.shape)

        if subsequent_fold:
            train_x = np.concatenate((features, loaded_features))
            train_y = np.concatenate((labels, loaded_labels))
        else:
            train_x = loaded_features
            train_y = loaded_labels
            subsequent_fold = True
        
    valid_fold_name = 'valid'
    feature_file = os.path.join(data_dir, valid_fold_name + '_x.npy')
    labels_file = os.path.join(data_dir, valid_fold_name + '_y.npy')
    valid_x = np.load(feature_file)
    valid_y = np.load(labels_file) 

    # and use the last fold for testing
    test_fold_name = 'test'
    feature_file = os.path.join(data_dir, test_fold_name + '_x.npy')
    labels_file = os.path.join(data_dir, test_fold_name + '_y.npy')
    test_x = np.load(feature_file)
    test_y = np.load(labels_file)
    
def load_folds(folds):
    subsequent_fold = False
    for k in range(len(folds)):
        #fold_name = 'fold' + str(folds[k])
        fold_name = fold_n[folds[k]]
        print(fold_name)
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        loaded_features = np.load(feature_file)
        loaded_labels = np.load(labels_file)
        print (fold_name, "features: ", loaded_features.shape)

        if subsequent_fold:
            features = np.concatenate((features, loaded_features))
            labels = np.concatenate((labels, loaded_labels))
        else:
            features = loaded_features
            labels = loaded_labels
            subsequent_fold = True
        
    return features, labels

<font color=blue size=4><b> 특성값 추출 </b></font>
- 특정 음성파일 입력 시 data point 및 특성값 결과 추출 예시

In [7]:
sample_filename = "data/Cut_Old_Men/5_oldman000.wav"
features = extract_feature_array(sample_filename)
data_points, _ = librosa.load(sample_filename)
print ("IN: Initial Data Points =", len(data_points))
print ("OUT: Total features =", np.shape(features))

IN: Initial Data Points = 27099
OUT: Total features = (1, 60, 41, 2)


<font color=blue size=4><b> train, valid, test용 audio npy 파일 생성 코드 (박)</b></font>
- 위 함수 호출
- fold_name 코드 수정 ( 이전, fold_name = 'fold' + str(k) )
- urban dataset 상에는 fold + 숫자 형태로 파일 저장
- train, valid, test 폴더로 구분하여 특성, 레이블 값 추출
- 레이블 : 여 0, 남 1, 어린이 2, 노인_여 3, 노인_남, 4

In [8]:
fold_n = ['train', 'valid', 'test']

In [53]:
# 입력 / 저장 경로 설정 및 데이터 변환 음성 -> .npz
parent_dir = "data/dataset/"  
save_dir = "data/dataset_npy"
assure_path_exists(save_dir)
save_folds(save_dir)


Saving train
Features of train  =  (12480, 60, 41, 2)
Labels of train  =  (12480, 5)
Saved data/dataset_npy/train_x.npy
Saved data/dataset_npy/train_y.npy

Saving valid
Features of valid  =  (2809, 60, 41, 2)
Labels of valid  =  (2809, 5)
Saved data/dataset_npy/valid_x.npy
Saved data/dataset_npy/valid_y.npy

Saving test
Features of test  =  (1974, 60, 41, 2)
Labels of test  =  (1974, 5)
Saved data/dataset_npy/test_x.npy
Saved data/dataset_npy/test_y.npy


<font color=blue size=4><b> 음성 분류용 CNN 모델</b></font>

In [53]:
tf.set_random_seed(0)
np.random.seed(0)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, LSTM, GRU
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from keras.utils import np_utils
#from keras.layers.extra import TimeDistributedConvolution2D, TimeDistributedMaxPooling2D, TimeDistributedFlatten

frames = 41
bands = 60
feature_size = bands * frames #60x41
num_labels = 5
num_channels = 2

In [72]:
def build_model():
    
    model = Sequential()   
    # input: 60x41 data frames, 2 channels => (60,41,2) tensors

    # filters of size 1x1 
    f_size = 1

    model.add(Convolution2D(48, f_size, strides=f_size, kernel_initializer='normal', padding='same', input_shape=(bands, frames, num_channels)))
    model.add(Convolution2D(48, f_size, strides=f_size, kernel_initializer='normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Convolution2D(96, f_size, strides=f_size, kernel_initializer='normal', padding='same'))
    model.add(Convolution2D(96, f_size, strides=f_size, kernel_initializer='normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # flatten output into a single dimension 
    # Keras will do shape inference automatically
    model.add(Flatten())

    # then a fully connected NN layer
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # finally, an output layer with one node per class
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))

    # use the Adam optimiser
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    
    return model 

### 평가함수

In [73]:
def evaluate(model):
    y_prob = model.predict_proba(test_x, verbose=0)
    y_pred = y_prob.argmax(axis=-1)
    y_true = np.argmax(test_y, 1)

    roc = roc_auc_score(test_y, y_prob)
    print ("ROC:",  round(roc,3))

    # evaluate the model
    score, accuracy = model.evaluate(test_x, test_y, batch_size=32)
    print("\nAccuracy = {:.2f}".format(accuracy))

    p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
    print ("F-Score:", round(f,2))
    
    return roc, accuracy

<font color=blue size=4><b> 생성한 .npz 파일 로드 Train, Valid, Test 데이터로 구분 </b></font>

In [88]:
data_dir = "data/dataset_npy/"

train_x, train_y = load_folds([0])

valid_x, valid_y = load_folds([1])
    
test_x, test_y = load_folds([2])

print("Building model...")
model = build_model()

earlystop = EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')

print("Training model...")
model.fit(train_x, train_y, validation_data=(valid_x, valid_y),  batch_size=20, epochs=10)
# validation에 트레이닝 시 10개 클래스를 했으면 validataion 및 test 데이터에도 10개 클래스 데이터가 있어야 함
#model.fit(train_x, train_y, callbacks=[earlystop], batch_size=20, epochs=10)

print("Evaluating model...")
roc, acc = evaluate(model)

train
train features:  (12480, 60, 41, 2)
valid
valid features:  (2809, 60, 41, 2)
test
test features:  (1974, 60, 41, 2)
Building model...
Training model...
Train on 12480 samples, validate on 2809 samples
Epoch 1/10
12480/12480 [==============================] - 37s 3ms/step - loss: 0.8115 - acc: 0.6846 - val_loss: 1.7924 - val_acc: 0.4315
Epoch 2/10
12480/12480 [==============================] - 36s 3ms/step - loss: 0.5816 - acc: 0.7896 - val_loss: 2.8832 - val_acc: 0.3895
Epoch 3/10
12480/12480 [==============================] - 36s 3ms/step - loss: 0.5243 - acc: 0.8096 - val_loss: 2.2756 - val_acc: 0.3667
Epoch 4/10
12480/12480 [==============================] - 36s 3ms/step - loss: 0.4791 - acc: 0.8309 - val_loss: 2.1861 - val_acc: 0.4105
Epoch 5/10
12480/12480 [==============================] - 36s 3ms/step - loss: 0.4412 - acc: 0.8419 - val_loss: 2.2109 - val_acc: 0.3841
Epoch 6/10
12480/12480 [==============================] - 36s 3ms/step - loss: 0.4035 - acc: 0.8588 - val_lo

<font color=blue size=4><b> 음성 예측 (남-여)</b></font>
- 현재 데이터는 남자 여자로만 레이블링
- 데이터 확보 후 변경

In [89]:
import IPython.display as ipd
import soundfile as sf
parent_dir = 'samples/voice/'
sound_file_paths = ["man_1.mp3","man_2.mp3","man_3.mp3","man_4.mp3","man_5k.mp3", "man_6k.mp3", "man_7k.mp3", "man_8.mp3",
                    "woman_1.mp3","woman_2.mp3","woman_3k.mp3","woman_4k.mp3","woman_5.mp3", "woman_6k.mp3","2_child_177.flac",
                    "2_child_206.flac","2_child_210.flac","3_oldwoman_211.flac","3_oldwoman_295.flac","3_oldwoman_302.flac",
                    "4_oldman245.flac","4_oldman249.flac","4_oldman250.flac","4_oldman279.flac"]

#ipd.Audio(parent_dir+sound_file_paths[4])

In [91]:
sound_names = ["남성1","남성2","남성3","남성4","남성5", "남성6", "남성7", "남성8",
               "여성1","여성2","여성3","여성4","여성5", "여성6","어린이1","어린이2","어린이3",
               "노_여성1","노_여성2","노_여성3","노_남성1","노_남성2","노_남성3","노_남성4"]

rst_value = ["여성", "남성", "어린이","노_여성","노_남성"]


for s in range(len(sound_names)):

    print ("\n----- ", sound_names[s], "-----")
    
    predict_file = parent_dir + sound_file_paths[s]
    predict_x = extract_feature_array(predict_file)
    print(predict_file)
    predictions = model.predict(predict_x)
    
    if len(predictions) == 0: 
        print ("No prediction")
        continue
    
    #for i in range(len(predictions[0])):
    #    print (sound_names[i], "=", round(predictions[0,i] * 100, 1))
    #print(predictions)
    
    ind = np.argpartition(predictions[0], -2)[-2:] # 큰값 상위 2개의 index
    #print(predictions)
    ind[np.argsort(predictions[0][ind])]
    ind = ind[::-1]
    #print(ind)
    
    print ("1순위 예측 (점수): ", rst_value[ind[0]], " (",round(predictions[0,ind[0]],3),")")
    print ("2순위 예측 (점수): ", rst_value[ind[1]], " (",round(predictions[0,ind[1]],3),")")


-----  남성1 -----
samples/voice/man_1.mp3
1순위 예측 (점수):  남성  ( 0.992 )
2순위 예측 (점수):  여성  ( 0.008 )

-----  남성2 -----
samples/voice/man_2.mp3
1순위 예측 (점수):  남성  ( 0.999 )
2순위 예측 (점수):  여성  ( 0.001 )

-----  남성3 -----
samples/voice/man_3.mp3
1순위 예측 (점수):  남성  ( 0.876 )
2순위 예측 (점수):  여성  ( 0.12 )

-----  남성4 -----
samples/voice/man_4.mp3
1순위 예측 (점수):  남성  ( 0.506 )
2순위 예측 (점수):  여성  ( 0.279 )

-----  남성5 -----
samples/voice/man_5k.mp3
1순위 예측 (점수):  어린이  ( 0.798 )
2순위 예측 (점수):  남성  ( 0.176 )

-----  남성6 -----
samples/voice/man_6k.mp3
1순위 예측 (점수):  어린이  ( 0.424 )
2순위 예측 (점수):  노_여성  ( 0.36 )

-----  남성7 -----
samples/voice/man_7k.mp3
1순위 예측 (점수):  어린이  ( 0.911 )
2순위 예측 (점수):  남성  ( 0.044 )

-----  남성8 -----
samples/voice/man_8.mp3
1순위 예측 (점수):  남성  ( 0.909 )
2순위 예측 (점수):  여성  ( 0.088 )

-----  여성1 -----
samples/voice/woman_1.mp3
1순위 예측 (점수):  여성  ( 0.954 )
2순위 예측 (점수):  어린이  ( 0.025 )

-----  여성2 -----
samples/voice/woman_2.mp3
1순위 예측 (점수):  남성  ( 0.858 )
2순위 예측 (점수):  여성  ( 0.136 )

-----  여

In [87]:
model.save('model/model_2epoch_CNN_2Dense.h5')